## Import packages

In [2]:
# Standard libraries
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2

# for env variables
import os
from dotenv import load_dotenv, get_key
load_dotenv()

# save env variables
SUPABASE_USER = get_key('.env', 'SUPABASE_USER')
SUPABASE_HOST = get_key('.env', 'SUPABASE_HOST')
SUPABASE_PASSWORD = get_key('.env', 'SUPABASE_PASSWORD')
SUPABASE_PORT = get_key('.env', 'SUPABASE_PORT')
SUPABASE_DB = get_key('.env', 'SUPABASE_DB')

## Pull fixtures and team data from the API 

In [3]:
# FPL API for fixtures
url = 'https://fantasy.premierleague.com/api/fixtures/'
response = requests.get(url)
fixtures_json = response.json()

# store in pandas DF
fixtures_df = pd.DataFrame(fixtures_json)
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2367698,NaN,False,False,162,None,0,False,None,12,NaN,3,NaN,[],2,2,93482
1,2367713,NaN,False,False,176,None,0,False,None,4,NaN,13,NaN,[],3,5,93496
2,2367538,1.0,True,True,1,2023-08-11T19:00:00Z,90,False,True,13,3.0,6,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,93321
3,2367540,1.0,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,93322
4,2367539,1.0,True,True,3,2023-08-12T14:00:00Z,90,False,True,19,1.0,3,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93323


In [4]:
# FPL API for teams
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()

# storing json outputs as dataframes
teams_df = pd.DataFrame(json['teams'])
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1230,1285,1250,1250,1210,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1115,1175,1130,1190,1100,1160,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1060,1095,1050,1100,1060,1090,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1125,1205,1120,1220,1130,1190,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1165,1210,1120,1200,1210,1240,131


## Join the tables to get team names into the fixtures table

In [5]:
# getting team names into fixtures table
fixtures_df['home_team'] = pd.merge(fixtures_df, teams_df, left_on='team_h', right_on='id', how='left')['name']
fixtures_df['away_team'] = pd.merge(fixtures_df, teams_df, left_on='team_a', right_on='id', how='left')['name']

fixtures_df[fixtures_df['finished'] == True].head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
2,2367538,1.0,True,True,1,2023-08-11T19:00:00Z,90,False,True,13,3.0,6,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,93321,Burnley,Man City
3,2367540,1.0,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,93322,Arsenal,Nott'm Forest
4,2367539,1.0,True,True,3,2023-08-12T14:00:00Z,90,False,True,19,1.0,3,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93323,Bournemouth,West Ham
5,2367541,1.0,True,True,4,2023-08-12T14:00:00Z,90,False,True,12,1.0,5,4.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,3,93324,Brighton,Luton
6,2367542,1.0,True,True,5,2023-08-12T14:00:00Z,90,False,True,10,1.0,9,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93325,Everton,Fulham


## Retain only necessary columns

In [6]:
# removing unnecessary columns
fixtures_df = fixtures_df[['id', 'event', 'finished', 'kickoff_time', 'team_a', 'team_a_score', 'team_h', 'team_h_score',
                           'team_h_difficulty', 'team_a_difficulty', 'home_team', 'away_team']]

fixtures_df.head()

,id,event,finished,kickoff_time,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,home_team,away_team
0,162,NaN,False,None,12,NaN,3,NaN,2,2,Bournemouth,Luton
1,176,NaN,False,None,4,NaN,13,NaN,3,5,Man City,Brentford
2,1,1.0,True,2023-08-11T19:00:00Z,13,3.0,6,0.0,5,2,Burnley,Man City
3,2,1.0,True,2023-08-12T12:00:00Z,16,1.0,1,2.0,2,4,Arsenal,Nott'm Forest
4,3,1.0,True,2023-08-12T14:00:00Z,19,1.0,3,1.0,2,2,Bournemouth,West Ham


In [7]:
fixtures_df = fixtures_df.rename(columns={'id': 'match_id', 'event': 'gameweek', 'team_a': 'away_team_id', 'team_h': 'home_team_id'})
fixtures_df.head()

,match_id,gameweek,finished,kickoff_time,away_team_id,team_a_score,home_team_id,team_h_score,team_h_difficulty,team_a_difficulty,home_team,away_team
0,162,NaN,False,None,12,NaN,3,NaN,2,2,Bournemouth,Luton
1,176,NaN,False,None,4,NaN,13,NaN,3,5,Man City,Brentford
2,1,1.0,True,2023-08-11T19:00:00Z,13,3.0,6,0.0,5,2,Burnley,Man City
3,2,1.0,True,2023-08-12T12:00:00Z,16,1.0,1,2.0,2,4,Arsenal,Nott'm Forest
4,3,1.0,True,2023-08-12T14:00:00Z,19,1.0,3,1.0,2,2,Bournemouth,West Ham


## Load into Supabase

In [8]:
# establish connection
conn = psycopg2.connect(
    database=SUPABASE_DB, 
    user=SUPABASE_USER, 
    password=SUPABASE_PASSWORD, 
    host=SUPABASE_HOST, 
    port=SUPABASE_PORT
)

# Setting auto commit true
conn.autocommit = True

# Creating a cursor object using the cursor() method
cursor = conn.cursor()


In [9]:
# Drop the table before creating it again
drop_query = 'DROP TABLE IF EXISTS public.dim_fpl_fixtures;'
cursor.execute(drop_query)

# close the cursor and connection
cursor.close()
conn.close()

In [10]:
# Create SQL alchemy engine
engine_url = 'postgresql://' + SUPABASE_USER + ':' + SUPABASE_PASSWORD + '@' + SUPABASE_HOST + '/' + SUPABASE_DB
engine = create_engine(engine_url)

# Load the table in supabase
fixtures_df.to_sql(
    'dim_fpl_fixtures',
    engine,
    schema='public',
    index=False
)

380